# Analisis de propeidades estaticas
- Energía cinética media -> Temperatura media
- Fuerza cuadratica media
- Presión
- ...

## Importación de librerías y lectura de datos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import os
os.getcwd()

'/Users/xaime/Desktop/Cuarto/Simulacion/Programas/Proyectos/Notebooks'

In [3]:
from pathlib import Path
#codigo ="110-160"

ruta_base = Path("..") / "resultados" / "dinamica"

ruta=ruta_base
print(ruta.exists())  # True o False
print(ruta.parent)    # Ruta al directorio padre
print(ruta.name)      # archivo.txt
print(ruta.stem)      # archivo

True
../resultados
dinamica
dinamica


In [4]:
#pd.read_csv(ruta, sep=r'\s+')

In [5]:
codigos=["110-160","160-210","110-160","210-260","260-310","310-360","360-410","410-460","460-510","510-560","560-610"] #Tiempo de inicio de simulacion-Tiempo de fin de simulacion
#codigos=["110-160","160-210"]
DFS_vx=[]
DFS_vy=[]
DFS_vz=[]

DATOS=[]

for codigo in codigos:
    DFS_vx.append(pd.read_csv(ruta_base / f"Simulacion_{codigo}"/"velocidad_x.txt", sep=r"\s+", header=None))
    DFS_vy.append(pd.read_csv(ruta_base/ f"Simulacion_{codigo}"/"velocidad_y.txt", sep=r"\s+", header=None))
    DFS_vz.append(pd.read_csv(ruta_base / f"Simulacion_{codigo}"/"velocidad_z.txt", sep=r"\s+", header=None))
    DATOS.append(pd.read_csv(ruta_base/ f"Simulacion_{codigo}"/"gname2.txt",skiprows=1, sep=r"\s+", header=None, names=['t','Ec','Ep','E_tot','W','T','P_corr','F2',"df_V","df_VV"]))


    

In [6]:
tiempos=[df.t.to_numpy() for df in DATOS]

tiempos[0]

array([1.000e-02, 2.000e-02, 3.000e-02, ..., 4.998e+01, 4.999e+01,
       5.000e+01], shape=(5000,))

In [7]:
pd.read_csv(ruta_base/ f"Simulacion_{codigo}"/"gname2.txt",skiprows=1, sep=r"\s+", header=None, names=['t','Ec','Ep','E_tot','W','T','P_corr','F2',"df_V","df_VV"]).head()

,t,Ec,Ep,E_tot,W,T,P_corr,F2,df_V,df_VV
0,0.01,984.9776,-1649.988,-665.0102,-789.5741,1.313304,0.359951,685.9454,0.296700,0.011123
1,0.02,987.5844,-1652.602,-665.0176,-836.4202,1.316779,0.346074,696.8760,0.312316,0.011035
2,0.03,994.5876,-1659.602,-665.0140,-953.2871,1.326117,0.311787,678.2810,0.351271,0.010818
3,0.04,1005.8390,-1670.855,-665.0161,-1131.5710,1.341119,0.259860,601.9847,0.410699,0.010492
4,0.05,1016.8950,-1681.908,-665.0133,-1304.7530,1.355860,0.209503,531.0198,0.468426,0.010176


### Formato de datos importados

- En las columnas tenemos las particulas de la caja etiquetadas del 0 al 499
- En las filas tenemos instantes sucesivos de tiempo grabados de nuestra simulacion.

In [8]:
DATOS[-1].iloc[:,1:].agg(["mean","std"])
#DATOS[-1].min()

,Ec,Ep,E_tot,W,T,P_corr,F2,df_V,df_VV
mean,1020.838556,-1685.862903,-665.024343,-1237.278831,1.361118,0.234624,572.803022,0.445935,0.010365
std,17.044929,17.047605,0.025719,337.659963,0.022727,0.106306,79.605908,0.112553,0.000717


In [34]:
F2s_mean=[]
F2s_std=[]

Ecs_mean=[]
Ecs_std=[]

Ec_1s_mean=[]
Ec_1s_std=[]

df_V_Ec_1s_mean=[]
df_V_Ec_1s_std=[]

df_V2_Ec_1s_mean=[]
df_V2_Ec_1s_std=[]

Eps_mean=[]
Eps_std=[]

Es_mean=[]
Es_std=[]

Ws_mean=[]
Ws_std=[]

df_Vs_mean=[]
df_Vs_std=[]

df_VVs_mean=[]
df_VVs_std=[]


Ps_mean=[]
Ps_std=[]

Ts_mean=[]
Ts_std=[]

for df in DATOS:
    df["Ec_1"] = 1 / df["Ec"]
    df["df_V_Ec_1"]= df["df_V"]*df["Ec_1"]
    df["df_V2_Ec_1"]= df["df_V"]**2 * df["Ec_1"]


for df in DATOS:
    F2s_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","F2"])
    F2s_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","F2"])

    Ecs_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","Ec"])
    Ecs_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","Ec"])

    Ec_1s_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","Ec_1"])
    Ec_1s_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","Ec_1"])

    df_V_Ec_1s_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","df_V_Ec_1"])
    df_V_Ec_1s_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","df_V_Ec_1"])

    df_V2_Ec_1s_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","df_V2_Ec_1"])
    df_V2_Ec_1s_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","df_V2_Ec_1"])

    Eps_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","Ep"])
    Eps_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","Ep"])

    Es_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","E_tot"])
    Es_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","E_tot"])

    Ws_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","W"])
    Ws_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","W"])

    df_Vs_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","df_V"])
    df_Vs_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","df_V"])

    df_VVs_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","df_VV"])
    df_VVs_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","df_VV"])

    Ps_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","P_corr"])
    Ps_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","P_corr"])

    Ts_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","T"])
    Ts_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","T"])

################################################
F2s_mean=np.array(F2s_mean)
F2s_std=np.array(F2s_mean)

################################################
Ecs_mean=np.array(Ecs_mean)
Ecs_std=np.array(Ecs_std)

Ec_1s_mean=np.array(Ec_1s_mean)
Ec_1s_std=np.array(Ec_1s_std)
Ec_1_mean = np.mean(Ec_1s_mean)
Ec_1_err = np.std(Ec_1s_mean, ddof=1) / np.sqrt(len(Ec_1s_mean))

df_V_Ec_1s_mean=np.array(df_V_Ec_1s_mean)
df_V_Ec_1s_std=np.array(df_V_Ec_1s_std)
df_V_Ec_1_mean = np.mean(df_V_Ec_1s_mean)
df_V_Ec_1_err = np.std(df_V_Ec_1s_mean, ddof=1) / np.sqrt(len(df_V_Ec_1s_mean))

df_V2_Ec_1s_mean=np.array(df_V2_Ec_1s_mean)
df_V2_Ec_1s_std=np.array(df_V2_Ec_1s_std)
df_V2_Ec_1_mean = np.mean(df_V2_Ec_1s_mean)
df_V2_Ec_1_err = np.std(df_V2_Ec_1s_mean, ddof=1) / np.sqrt(len(df_V2_Ec_1s_mean))


################################################


"""
Calculamos los valores medios de cada simulación. Así podremos obtener todas las magnitudes termodinámicas con los datos de 1 simulación.
Y teniendo más simulaciones lo que haremos sera tener más "medidas" y asi poder sacar medias y desviaciones
"""

Eps_mean=np.array(Eps_mean)
Eps_std=np.array(Eps_std)

Es_mean=np.array(Es_mean)
Es_std=np.array(Es_std)
E_mean = np.mean(Es_mean)
E_err = np.std(Es_mean, ddof=1) / np.sqrt(len(Es_mean))


Ws_mean=np.array(Ws_mean)
Ws_std=np.array(Ws_std)

df_Vs_mean=np.array(df_Vs_mean)
df_Vs_std=np.array(df_Vs_std)

df_VVs_mean=np.array(df_VVs_mean)
df_VVs_std=np.array(df_VVs_std)

Ps_mean=np.array(Ps_mean)
Ps_std=np.array(Ps_std)


Ts_mean=np.array(Ts_mean)
Ts_std=np.array(Ts_std)


"""
Obtenemos las medias globales y las desviaciones estandar
Tomamos los valores de cada simulacion como 1 medida. Y si hemos simulado 10 veces pues tendremos 10 medias con su correspondiente media y desviación estandar
"""

# Media global = media de las medias
F2_mean = np.mean(F2s_mean)
Ec_mean = np.mean(Ecs_mean)


Ep_mean = np.mean(Eps_mean)



W_mean = np.mean(Ws_mean)
df_V_mean = np.mean(df_Vs_mean)
df_VV_mean = np.mean(df_VVs_mean)

P_mean  = np.mean(Ps_mean)
T_mean  = np.mean(Ts_mean)

# Error estándar de la media global --> ¿Deberia tener en cuenta las std de cada bloque simulado?
F2_err = np.std(F2s_mean, ddof=1) / np.sqrt(len(F2s_mean))
Ec_err = np.std(Ecs_mean, ddof=1) / np.sqrt(len(Ecs_mean))

Ep_err = np.std(Eps_mean, ddof=1) / np.sqrt(len(Eps_mean))
W_err = np.std(Ws_mean, ddof=1) / np.sqrt(len(Ws_mean))
df_V_err = np.std(df_Vs_mean, ddof=1) / np.sqrt(len(df_Vs_mean))
df_VV_err = np.std(df_VVs_mean, ddof=1) / np.sqrt(len(df_VVs_mean))
P_err  = np.std(Ps_mean, ddof=1) / np.sqrt(len(Ps_mean))
T_err  = np.std(Ts_mean, ddof=1) / np.sqrt(len(Ts_mean))



In [44]:
print(f"<F²> = {F2_mean:.6f} ± {F2_err:.6f}")
print(f"<Ec> = {Ec_mean:.6f} ± {Ec_err:.6f}")
print(f"<Ec_1> = {Ec_1_mean:.6f} ± {Ec_1_err:.6f}")
print(f"<df_V_Ec_1> = {df_V_Ec_1_mean:.6f} ± {df_V_Ec_1_err:.6f}")
print(f"<df_V2_Ec_1> = {df_V2_Ec_1_mean:.6f} ± {df_V2_Ec_1_err:.6f}")
print(f"<Ep> = {Ep_mean:.6f} ± {Ep_err:.6f}")
print(f"<W> = {W_mean/3000:.6f} ± {W_err/3000:.6f}")
print(f"<df_V> = {df_V_mean:.6f} ± {df_V_err:.6f}")
print(f"<df_VV> = {df_VV_mean:.6f} ± {df_VV_err:.6f}")

print(f"<P>  = {P_mean:.6f} ± {P_err:.6f}")
print(f"<T>  = {T_mean:.6f} ± {T_err:.6f}")

<F²> = 572.620316 ± 0.866832
<Ec> = 1023.669169 ± 0.502634
<Ec_1> = 0.000977 ± 0.000000
<df_V_Ec_1> = 0.000434 ± 0.000002
<df_V2_Ec_1> = 0.000205 ± 0.000001
<Ep> = -1688.693250 ± 0.503223
<W> = -0.411919 ± 0.001657
<df_V> = 0.445427 ± 0.001657
<df_VV> = 0.010392 ± 0.000013
<P>  = 0.237019 ± 0.001756
<T>  = 1.364892 ± 0.000670


# 🧮 Propiedades estáticas


## Ecuaciones teóricas (Error: estas ecuaciones son para la colectividad canonica, no la microcanonica)

$$
C_V = \frac{3}{2} + \frac{1}{T^2} \left( \langle \phi^2 \rangle - \langle \phi \rangle^2 \right)
$$

$$
\frac{1}{\kappa_T} = \frac{N T}{V} + V \langle \phi_{VV} \rangle - \frac{V}{T} \left( \langle \phi_V^2 \rangle - \langle \phi_V \rangle^2 \right)
$$

$$
\gamma = \frac{V}{C_V} 
\left[
\frac{N k_B}{V} + \frac{1}{k_B T^2}
\left(
\langle \phi \rangle \langle \phi_V \rangle - \langle \phi \phi_V \rangle
\right)
\right]
$$

---

## Para calcular todas las propiedades se necesita:
- $\langle E \rangle, \langle E^2 \rangle$
- $\langle \phi_V \rangle, \langle \phi_V^2 \rangle$
- $\langle \phi_{VV} \rangle$
- $\langle \phi \rangle, \langle \phi \phi_V \rangle$
- $T, N, V$


In [45]:
def Cv(f,Ec_mean, Ec_1_mean):
    return 1/( 1+ (2/f -1)*Ec_mean*Ec_1_mean )

def gamma(V, Cv, N,f, df_V_mean, Ec_1_mean, df_V_Ec_1_mean):
    return (N/Cv)+ V*(f/2-1)*( df_V_mean*Ec_1_mean - df_V_Ec_1_mean  )
    
def ks(N, T, V, f, gamma, Cv, df_VV_mean,df_V2_Ec_1_mean,df_V_mean,df_V_Ec_1_mean,Ec_1_mean):
    inv_ks=(N*T/V) * (1+2*gamma - N/Cv) + V*df_VV_mean - V*(f/2-1)*( df_V2_Ec_1_mean - 2*df_V_mean*df_V_Ec_1_mean + df_V_mean**2 * Ec_1_mean )
    return 1/inv_ks


def alpha_E(V,f,Ec_mean,df_V_Ec_1_mean,df_V_mean):
    return 1/( V*((1-2/f)*Ec_mean * df_V_Ec_1_mean - df_V_mean   )   )

def alpha_E_otra_formula(P,V,Cv,gamma,T):
    inv_alpha_E= P*V/Cv - gamma*T
    return 1/inv_alpha_E

def alpha_S(gamma,T):
    return -1/(gamma*T)
    

def kT(ks,T,Cv,gamma,V):
    inv_kT=1/ks - (T*Cv*gamma**2)/V
    return 1/inv_kT

def alpha_P(Cv,V,kT):
    return Cv/V * gamma * kT


def Cp(Cv,kT,ks):
    return Cv* kT/ks
    
    
    
#def alpha_P()

#def alpha_E


#def Cv(T, Ep2_mean, Ep_mean2, N):
 #   return 1.5 + (Ep2_mean - Ep_mean2) / T**2

#def Kt_1(N, T, V, df_VV_mean, df_V2_mean, df_V_mean2):
  #  return (N*T/V) + V*df_VV_mean - (V/T)*(df_V2_mean - df_V_mean2)

#def gamma(V, Cv_total, N, T, phi_mean, phiV_mean, phi_phiV_mean):
 #   return (V / Cv_total) * ((N / V) + ((phi_mean*phiV_mean - phi_phiV_mean) / (T**2)))

In [46]:
N=500
V=1000
f=3*500-3


Cv=Cv(f,Ec_mean, Ec_1_mean)
gamma=gamma(V, Cv, N,f, df_V_mean, Ec_1_mean, df_V_Ec_1_mean)

ks=ks(N, T_mean, V,f, gamma, Cv, df_VV_mean,df_V2_Ec_1_mean,df_V_mean,df_V_Ec_1_mean,Ec_1_mean)
kT=kT(ks,T_mean,Cv,gamma,V)

alpha_E=alpha_E(V,f,Ec_mean,df_V_Ec_1_mean,df_V_mean)
alpha_E_2=alpha_E_otra_formula(P_mean,V,Cv,gamma,T_mean)
alpha_S=alpha_S(gamma,T_mean)
alpha_P=alpha_P(Cv,V,kT)


Cp=Cp(Cv,kT,ks)


In [51]:
print("***********************************")
print("Variables Termodinámicas")
print(f"<N>  = 500")
print(f"<V>  = 1000")
print(f"<E>  = {E_mean:.6f} ± {E_err:.6f}")

print(f"<P>  = {P_mean:.6f} ± {P_err:.6f}")
print(f"<T>  = {T_mean:.6f} ± {T_err:.6f}")




print("***********************************")
print("Coeficientes de compresibilidad")
print(f"Cv = {Cv:.6}")
print(f"Cp = {Cp:.6}")

print("***********************************")
print("Gamma de Gruneisen")
print(f"gamma = {gamma:.6}")

print("***********************************")
print("Coeficientes de compresibilidad")
print(f"ks= {ks:.6}")
print(f"kt= {kT:.6}")

print("***********************************")
print("Coeficientes de dilatación")
print(f"alpha_E= {alpha_E:.6}")
print(f"alpha_E (otra formula)= {alpha_E_2:.6}")
print(f"alpha_P= {alpha_P:.6}")
print(f"alpha_S= {alpha_S:.6}")

print("***********************************")

***********************************
Variables Termodinámicas
<N>  = 500
<V>  = 1000
<E>  = -665.024081 ± 0.001485
<P>  = 0.237019 ± 0.001756
<T>  = 1.364892 ± 0.000670
***********************************
Coeficientes de compresibilidad
Cv = 973.145
Cp = 3924.49
***********************************
Gamma de Gruneisen
gamma = 1.34207
***********************************
Coeficientes de compresibilidad
ks= 0.314348
kt= 1.2677
***********************************
Coeficientes de dilatación
alpha_E= -0.628739
alpha_E (otra formula)= -0.629636
alpha_P= 1.65565
alpha_S= -0.545917
***********************************


In [49]:
3924.49/973.145

4.032790591330172

In [50]:
kT/ks

np.float64(4.032792044221464)